In [1]:
import configparser
import datetime 
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.functions import monotonically_increasing_id
import glob

In [2]:
#mount the blob in Azure storage account
dbutils.fs.mount(
  source = "wasbs://songdata@songdata.blob.core.windows.net",
  mount_point = "/mnt/song-data",
  extra_configs = {"fs.azure.account.key.songdata.blob.core.windows.net":dbutils.secrets.get(scope = "sc_songlog", key = "songblob")})


Out[ 3 ]: True

In [3]:
#Check the mount. Two blob containers are accessable.
%sh ls /dbfs/mnt/song-data

log-data
song-data

In [4]:
#A function to parse out all the file paths in each container
import re

def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [5]:
#Parse the container and edit each path to comply with DBFS syntax 
song_files=get_files("/dbfs/mnt/song-data/song-data")
song_files = list(map(lambda x: re.sub(r'\/dbfs','dbfs:',x), song_files))
#print(song_files)

In [6]:
#Define a songSchema for songdata
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
songSchema = R([
    Fld("artist_id",Str()),
    Fld("artist_latitude",Dbl()),
    Fld("artist_location",Str()),
    Fld("artist_longitude",Dbl()),
    Fld("artist_name",Str()),
    Fld("duration",Dbl()),
    Fld("num_songs",Int()),
    Fld("song_id",Str()),
    Fld("title",Str()),
    Fld("year",Int()),
])

In [7]:
#Load all the json files 
song_data = spark.read.format('json') \
  .option('schema','songSchema')\
  .load(song_files)


In [8]:
song_data.show(5)

+------------------+---------------+-----------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
 artist_id|artist_latitude| artist_location|artist_longitude| artist_name| duration|num_songs| song_id| title|year|
+------------------+---------------+-----------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
ARDR4AC1187FB371A1| null| | null|Montserrat Caball...|511.16363| 1|SOBAYLL12A8C138AF9|Sono andati? Fing...| 0|
AREBBGV1187FB523D2| null| Houston, TX| null|Mike Jones (Featu...|173.66159| 1|SOOLYAZ12A6701F4A6|Laws Patrolling (...| 0|
ARMAC4T1187FB3FA4C| 40.82624|Morris Plains, NJ| -74.47995|The Dillinger Esc...|207.77751| 1|SOBBUGU12A8C13E95D|Setting Fire to S...|2004|
ARPBNLO1187FB3D52F| 40.71455| New York, NY| -74.00712| Tiny Tim| 43.36281| 1|SOAOIBZ12AB01815BE|I Hold Your Hand ...|2000|
ARDNS031187B9924F0| 32.67828| Georgia| -83.22295| Tim Wilson|186.48771| 1|SONYPOM12A8C13B2D7|I Think My Wife I...|2005|
+------------------+---------------+-----------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
only showing top 5 rows

In [9]:
song_data.count()

Out[ 9 ]: 71

In [10]:
#Save as Hive table  
table_name = "song_data"
song_data.write.mode('OverWrite').format('parquet').saveAsTable(table_name)

In [11]:
%sh ls /dbfs/user/hive/warehouse

song_data

In [12]:
#Create a temp view that has Hive metadata for Spark SQL
song_data.createOrReplaceTempView("song_data")

In [13]:
# extract columns to create songs table
song_table = spark.sql("""
            SELECT distinct song_id, title, artist_id, year, duration
            from song_data
            order by year, artist_id
            """)

In [14]:
# write songs table to parquet files partitioned by year and artist
song_table.write.partitionBy("year","artist_id").parquet("data/output/song_table")

In [15]:
# extract columns to create artists table
artists_table = spark.sql("""
            SELECT distinct artist_id, 
            artist_name as name, 
            artist_location as location, 
            artist_latitude as latitude, 
            artist_longitude as longitude
            from song_data
            """)

In [16]:
# write artists table to parquet files
artists_table.write.parquet("data/output/artists_table")

In [17]:
#Get the paths for log-file
log_files=get_files("/dbfs/mnt/song-data/log-data")
log_files = list(map(lambda x: re.sub(r'\/dbfs','dbfs:',x), log_files))

In [18]:
#Load all the log json files
log_data = spark.read.json(log_files)
log_data.show(10)

+-----------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
 artist| auth|firstName|gender|itemInSession|lastName| length|level| location|method| page| registration|sessionId| song|status| ts| userAgent|userId|
+-----------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
 Harmonia| Logged In| Ryan| M| 0| Smith|655.77751| free|San Jose-Sunnyval...| PUT|NextSong|1.541016707796E12| 583| Sehr kosmisch| 200|1542241826796|"Mozilla/5.0 (X11...| 26|
The Prodigy| Logged In| Ryan| M| 1| Smith|260.07465| free|San Jose-Sunnyval...| PUT|NextSong|1.541016707796E12| 583| The Big Gundown| 200|1542242481796|"Mozilla/5.0 (X11...| 26|
 Train| Logged In| Ryan| M| 2| Smith|205.45261| free|San Jose-Sunnyval...| PUT|NextSong|1.541016707796E12| 583| Marry Me| 200|1542242741796|"Mozilla/5.0 (X11...| 26|
 null| Logged In| Wyatt| M| 0| Scott| null| free|Eureka-Arcata-For...| GET| Home|1.540872073796E12| 563| null| 200|1542247071796|Mozilla/5.0 (Wind...| 9|
 null| Logged In| Austin| M| 0| Rosales| null| free|New York-Newark-J...| GET| Home|1.541059521796E12| 521| null| 200|1542252577796|Mozilla/5.0 (Wind...| 12|
Sony Wonder| Logged In| Samuel| M| 0|Gonzalez|218.06975| free|Houston-The Woodl...| PUT|NextSong|1.540492941796E12| 597| Blackbird| 200|1542253449796|"Mozilla/5.0 (Mac...| 61|
 null| Logged In| Samuel| M| 1|Gonzalez| null| free|Houston-The Woodl...| GET| About|1.540492941796E12| 597| null| 200|1542253460796|"Mozilla/5.0 (Mac...| 61|
 null|Logged Out| null| null| 0| null| null| paid| null| PUT| Login| null| 602| null| 307|1542260074796| null| |
 null| Logged In| Tegan| F| 1| Levine| null| paid|Portland-South Po...| GET| Home|1.540794356796E12| 602| null| 200|1542260277796|"Mozilla/5.0 (Mac...| 80|
 Van Halen| Logged In| Tegan| F| 2| Levine|289.38404| paid|Portland-South Po...| PUT|NextSong|1.540794356796E12| 602|Best Of Both Worl...| 200|1542260935796|"Mozilla/5.0 (Mac...| 80|
+-----------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
only showing top 10 rows

In [19]:
log_data.count()

Out[ 44 ]: 8056

In [20]:
# filter by actions for song plays
df = log_data.filter(log_data.page=="NextSong")
df.count()

Out[ 45 ]: 6820

In [21]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M:%S'))
df = df.withColumn("start_time", get_timestamp(df.ts))

In [22]:
df.createOrReplaceTempView("log_data")

In [23]:
# extract columns for users table    
user_table = spark.sql("""
            SELECT distinct int(userId) as user_id, 
            firstName as first_name, 
            lastName as last_name, 
            gender,
            level
            from log_data
            """)

In [24]:
user_table.count()

Out[ 52 ]: 104

In [25]:
user_table.write.parquet("data/output/user_table")

In [26]:
# extract columns to create time table
time_table = spark.sql("""
            SELECT distinct start_time,
            hour(start_time) as hour   ,
            dayofmonth(start_time) as day,
            weekofyear(start_time) as week,
            month(start_time) as month,
            year(start_time) as year, 
            dayofweek(start_time) as weekday
            from log_data
            order by year, month
            """)

In [27]:
time_table.show(3)

+-------------------+----+---+----+-----+----+-------+
 start_time|hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
2018-11-15 11:20:26| 11| 15| 46| 11|2018| 5|
2018-11-15 12:28:39| 12| 15| 46| 11|2018| 5|
2018-11-15 15:59:09| 15| 15| 46| 11|2018| 5|
+-------------------+----+---+----+-----+----+-------+
only showing top 3 rows

In [28]:
# write time table to parquet files partitioned by year and month
time_table.write.parquet("data/output/time_table")

In [29]:
#Construct an object to transform the start_time to year and month formats 
import tempfile
from pyspark.sql.functions import col, dayofmonth, month, year

dt = col("start_time").cast("date")
fname = [(year, "year"), (month, "month")]
exprs = [col("*")] + [f(dt).alias(name) for f, name in fname]

In [30]:
songplays_table = spark.sql("""
          select monotonically_increasing_id() as songplay_id , 
                l.start_time, 
                l.userId as user_id, 
                l.level, 
                s.song_id, 
                a.artist_id, 
                l.sessionId as session_id, 
                l.location, 
                l.userAgent as user_agent
           from log_data l
           left join song_data s on l.song=s.title
           left join song_data a on l.artist=a.artist_name
           order by year(l.start_time), month(l.start_time)
""")

In [31]:
#Build the partition tables
songplays_table\
       .select(*exprs)\
       .write\
       .partitionBy(*(name for _, name in fname))\
       .format("parquet")\
       .save("data/output/songplays")

In [32]:
haha=spark.read.parquet("data/output/songplays/year=2018/month=11/*")

In [33]:
haha.show(5)

+-----------+-------------------+-------+-----+-------+---------+----------+--------------------+--------------------+
songplay_id| start_time|user_id|level|song_id|artist_id|session_id| location| user_agent|
+-----------+-------------------+-------+-----+-------+---------+----------+--------------------+--------------------+
 0|2018-11-15 00:30:26| 26| free| null| null| 583|San Jose-Sunnyval...|"Mozilla/5.0 (X11...|
 1|2018-11-15 00:41:21| 26| free| null| null| 583|San Jose-Sunnyval...|"Mozilla/5.0 (X11...|
 2|2018-11-15 00:45:41| 26| free| null| null| 583|San Jose-Sunnyval...|"Mozilla/5.0 (X11...|
 3|2018-11-15 03:44:09| 61| free| null| null| 597|Houston-The Woodl...|"Mozilla/5.0 (Mac...|
 4|2018-11-15 05:48:55| 80| paid| null| null| 602|Portland-South Po...|"Mozilla/5.0 (Mac...|
+-----------+-------------------+-------+-----+-------+---------+----------+--------------------+--------------------+
only showing top 5 rows